In [1]:
import numpy as np
import bokeh.plotting as blt
import bokeh
bokeh.io.output_notebook()
import Cython
%load_ext Cython
%load_ext line_profiler
%load_ext autoreload
%autoreload 2

Loading BokehJS ...

In [2]:
import causticTools.sim as cSim
import causticTools.io as cIo
import causticTools.analytic as cAnalytic
import pyximport
pyximport.install()
from causticTools.cythonSim import massFunction

In [16]:
M=1e9
m=9e8
rho=1
vFactor=(4/3)*np.pi
rho=1
class baryonInit(massFunction):
    def evaluate(self,r):
        return M
class baryonMass(massFunction):
    def evaluate(self,r):
        return m
class dmMass(massFunction):
    def evaluate(self,r):
        return vFactor*rho*(r**3)
class findEcc(massFunction):
    def evaluate(self,P):
        return 0.25#1-np.sqrt(1-P)

In [64]:
nShells=32
nPhase=1
nEcc=1
T=1e7
dt=1e5
rMin=50
rMax=500
name='Kepler.r'+str(nShells)+'p'+str(2*nPhase)+'e'+str(nEcc)
nOutput=4
cSim.runSim(nShells,nPhase,nEcc,T,dt,rMin,rMax,name,nOutput,dmMass(),baryonInit(),baryonMass(),findEcc())

(66, ' edges')
(64, ' shells')
('Saving to file: ', 'output/Kepler.r32p2e1.0_4.txt')
('Saving to file: ', 'output/Kepler.r32p2e1.1_4.txt')
('Saving to file: ', 'output/Kepler.r32p2e1.2_4.txt')
('Saving to file: ', 'output/Kepler.r32p2e1.3_4.txt')


In [65]:
rMin=50
rMax=500
dR=rMax-rMin
nOutput=4
figs=[]
for j in range(1,6):
    fig=blt.figure(x_range=(rMin-0.1*dR,rMax+0.1*dR),y_range=(0,5),width=500,height=250)
    for i in range(nOutput):
        fig=cIo.plotSmoothDens(fig,i,nOutput,'Kepler.r32p'+str(2**j)+'e1',1,rMin,rMax)
    figs.append(fig)
print(figs)
grid=bokeh.layouts.gridplot([figs])
blt.show(grid)

[Figure(id='abebca55-b766-4583-84f2-8602ab29a404', ...), Figure(id='3a3644f3-b3f6-4d76-9ad9-e67087ba0a91', ...), Figure(id='90222558-0237-4fc3-9ff8-91b34921d9dc', ...), Figure(id='67ff5f6f-c168-43f0-961f-d4f6656487b1', ...), Figure(id='7078f3d7-62c6-4f51-87e6-6bc17734dd91', ...)]


In [19]:
rMin=50
rMax=500
dR=rMax-rMin
nOutput=4
fig=blt.figure(x_range=(rMin-0.1*dR,rMax+0.1*dR),y_range=(0,5))
for i in range(nOutput):
    fig=cIo.plotDens(fig,i,nOutput,'1phase')
blt.show(fig)

In [9]:
aR,aDens=cAnalytic.densProfile(rMin,rMax,128,M,m,7.5e6)

/Users/Zephyr/Google Drive/astro/caustics/causticTools/analytic.py:37: RuntimeWarning: divide by zero encountered in double_scalars
  return ((G*m*(t**2))*((1-eps*np.cos(eta))**3)/((eta-eps*np.sin(eta))**2))**(1/3)


In [10]:
dR=rMax-rMin
fig=blt.figure(x_range=(rMin-0.1*dR,rMax+0.1*dR),y_range=(0,5))
fig=cIo.plotDensProfile(fig,aR,aDens,rMin,rMax,0.5)
fig=cIo.plotDens(fig,3,nOutput,'1phase')
fig=cIo.plotSmoothDens(fig,3,nOutput,'1phase',1,rMin,rMax)
blt.show(fig)

In [6]:
%%cython
cdef class Function:
    cpdef double evaluate(self, double x) except *:
        return 0

cdef Function globalFunction
cdef Function globalFunction2
    
cpdef setFunction(Function f, Function f2):
    global globalFunction, globalFunction2
    globalFunction=f
    globalFunction2=f2

cpdef printFunction(double r):
    print(globalFunction2.evaluate(r))

def integrate(Function f, double a, double b, int N):
    cdef int i
    cdef double s, dx
    if f is None:
        raise ValueError("f cannot be None")
    s = 0
    dx = (b-a)/N
    for i in range(N):
        s += f.evaluate(a+i*dx)
    return s * dx

In [7]:
class MyPolynomial():
    def evaluate(self, x):
        return 2*x*x + 3*x - 12
    
class MyPolynomial2():
    def evaluate(self, x):
        return 2*x*x + 3*x - 6

integrate(MyPolynomial(), 0, 1, 10000)

setFunction(MyPolynomial(),MyPolynomial2())
printFunction(1)

TypeError: Argument 'f' has incorrect type (expected _cython_magic_b914fad814b3f57d6899ec1f1b6a9fa7.Function, got MyPolynomial)